In [1]:
from keras.models import Model
from pickle import load
from keras import layers
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint  
import matplotlib.pyplot as plt
%matplotlib inline

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [27]:
# this was for vggface model with output (7, 7, 2048), i also tried models with (2048) and (4096) output 
# and built model with only dense layers and dropout 
in1 = layers.Input(shape=(7, 7, 2048))

X = layers.Conv2D(filters=1024, kernel_size=3, padding='same')(in1)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)

X = layers.MaxPooling2D(pool_size=2)(X)

X = layers.Conv2D(filters=1024, kernel_size=3, padding='same')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)

X = layers.MaxPooling2D(pool_size=2)(X)

X = layers.Flatten()(X)
X = layers.Dropout(0.3)(X)
X = layers.Dense(512, activation='relu')(X) 
pred = layers.Dense(10,activation='softmax')(X)
model = Model(inputs=in1, outputs=pred)

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 7, 7, 2048)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 1024)        18875392  
_________________________________________________________________
batch_normalization_9 (Batch (None, 7, 7, 1024)        4096      
_________________________________________________________________
activation_9 (Activation)    (None, 7, 7, 1024)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 3, 3, 1024)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 3, 1024)        9438208   
_________________________________________________________________
batch_normalization_10 (Batc (None, 3, 3, 1024)        4096      
__________

### I tried many optimizers, SGD seems to work best with this dataset with lr = 0.1 nad momenturm 0.8
### I tried adam and rmsprop but they get stuck or too slow ( need epochs > 64 ) and still val_acc doesn't increase

In [29]:
from keras.optimizers import SGD

In [31]:
opt = SGD(lr = 0.15, momentum=0.8)

In [32]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

### use the trainig data we created, first 180 file for training and [181,185] for validating

In [33]:
def gen():
    i = 0
    while True:
        i = i + 1
        x,y = load(open('vggtrain/train' + str(i) + '.pkl', 'rb'))
        yield x,y
        if i == 180:
            i = 0

In [34]:
def gen2():
    i = 180
    while True:
        i = i + 1
        x,y = load(open('vggtrain/train' + str(i) + '.pkl', 'rb'))
        yield x,y
        if i == 185:
            i = 180

### fit the model

In [ ]:
hist = model.fit_generator(epochs=32, generator=gen(),steps_per_epoch=180,validation_data=gen2(),validation_steps=5)

Epoch 1/32
180/180 [==============================] - 1735s 10s/step - loss: 2.8279 - acc: 0.1756 - val_loss: 2.1102 - val_acc: 0.2134
Epoch 2/32
180/180 [==============================] - 1828s 10s/step - loss: 2.0801 - acc: 0.2208 - val_loss: 2.0719 - val_acc: 0.2271
Epoch 3/32
180/180 [==============================] - 1904s 11s/step - loss: 2.0613 - acc: 0.2285 - val_loss: 2.0631 - val_acc: 0.2296
Epoch 4/32
180/180 [==============================] - 1901s 11s/step - loss: 2.0472 - acc: 0.2341 - val_loss: 2.0519 - val_acc: 0.2335
Epoch 5/32
 91/180 [==============>...............] - ETA: 14:04 - loss: 2.0361 - acc: 0.2383

In [ ]:
hist.history.keys()

In [23]:
def gen3():
    i = 95
    while True:
        i = i + 1
        x,y = load(open('vggtrain/train' + str(i) + '.pkl', 'rb'))
        yield x,y
        if i == 105:
            i = 95

### evaluate on current model files [96,105]

In [24]:
a,b = model.evaluate_generator(generator=gen3(),steps=10)